In [44]:
from splinter import Browser
from bs4 import BeautifulSoup as bs
import time
import requests
import pandas as pd


In [45]:
def init_browser():
    # @NOTE: Replace the path with your actual path to the chromedriver
    executable_path = {"executable_path": "/usr/local/bin/chromedriver"}
    return Browser("chrome", **executable_path, headless=False)



In [ ]:
def scrape_info():

    #run above function 'init_browser'
    browser = init_browser()

    # Scrape the NASA Mars News Site and collect the latest News
    url = "https://mars.nasa.gov/news/?page=0&per_page=40&order=publish_date+desc%2Ccreated_at+desc&search=&category=19%2C165%2C184%2C204&blank_scope=Latest"
    browser.visit(url)

    time.sleep(1)

    # Scrape page into Soup
    html = browser.html
    soup = bs(html, "html.parser")

    # Get the News Info Section
    news_info = soup.find('div', class_='list_text')
    #print(news_info.text)

    # collect the latest News Title
    news_sec1 = news_info.find('div', class_='content_title')
    news_title = news_sec1.a.text
    #print(news_title)

    # news article Paragraph Text
    news_p = news_info.find('div', class_="article_teaser_body").text
    #print(news_p)

    # Close the browser after scraping
    browser.quit()

In [ ]:
    ### LETS GET THE JPL IMAGE!!
    # Visit the url for JPL Featured Space Image here

    #run above function 'init_browser'
    browser = init_browser()

    url = "https://www.jpl.nasa.gov/spaceimages/?search=&category=Mars"
    browser.visit(url)

    time.sleep(5)

    # Scrape page into Soup
    html = browser.html
    soup = bs(html, "html.parser")


    # Use splinter to click on main 'full image' button
    image_button = browser.find_by_id('full_image')

    image_button.click()
    time.sleep(1)

    # Clicking 'more info' button. Making way to large image info
    browser.find_by_css(".buttons .button").click()
    time.sleep(1)

    #image url for the current Featured Mars Image
    browser.find_by_css(".lede").click()
    time.sleep(1)

    #assign the url string to a variable called featured_image_url
    featured_image_url = browser.url

    # Close the browser after scraping
    browser.quit()

In [32]:
### LETS GET THE MARS FUN FACTS!!

#run above function 'init_browser'
browser = init_browser()

url = "https://space-facts.com/mars/"
browser.visit(url)

time.sleep(5)

# Scrape page into Soup
html = browser.html
soup = bs(html, "html.parser")

In [33]:
tables = pd.read_html(url)

In [34]:
# Close the browser after scraping
browser.quit()

In [35]:
df = tables[0]
## remove index
df.set_index([0], inplace=True)
df

,1
0,
Equatorial Diameter:,"6,792 km"
Polar Diameter:,"6,752 km"
Mass:,6.39 × 10^23 kg (0.11 Earths)
Moons:,2 (Phobos & Deimos)
Orbit Distance:,"227,943,824 km (1.38 AU)"
Orbit Period:,687 days (1.9 years)
Surface Temperature:,-87 to -5 °C
First Record:,2nd millennium BC
Recorded By:,Egyptian astronomers


In [37]:
df.columns.name = None
df

,1
0,
Equatorial Diameter:,"6,792 km"
Polar Diameter:,"6,752 km"
Mass:,6.39 × 10^23 kg (0.11 Earths)
Moons:,2 (Phobos & Deimos)
Orbit Distance:,"227,943,824 km (1.38 AU)"
Orbit Period:,687 days (1.9 years)
Surface Temperature:,-87 to -5 °C
First Record:,2nd millennium BC
Recorded By:,Egyptian astronomers


In [39]:
## HTML tables from DataFrames
mars_facts = df.to_html()
mars_facts

'<table border="1" class="dataframe">\n  <thead>\n    <tr style="text-align: right;">\n      <th></th>\n      <th>1</th>\n    </tr>\n    <tr>\n      <th>0</th>\n      <th></th>\n    </tr>\n  </thead>\n  <tbody>\n    <tr>\n      <th>Equatorial Diameter:</th>\n      <td>6,792 km</td>\n    </tr>\n    <tr>\n      <th>Polar Diameter:</th>\n      <td>6,752 km</td>\n    </tr>\n    <tr>\n      <th>Mass:</th>\n      <td>6.39 × 10^23 kg (0.11 Earths)</td>\n    </tr>\n    <tr>\n      <th>Moons:</th>\n      <td>2 (Phobos &amp; Deimos)</td>\n    </tr>\n    <tr>\n      <th>Orbit Distance:</th>\n      <td>227,943,824 km (1.38 AU)</td>\n    </tr>\n    <tr>\n      <th>Orbit Period:</th>\n      <td>687 days (1.9 years)</td>\n    </tr>\n    <tr>\n      <th>Surface Temperature:</th>\n      <td>-87 to -5 °C</td>\n    </tr>\n    <tr>\n      <th>First Record:</th>\n      <td>2nd millennium BC</td>\n    </tr>\n    <tr>\n      <th>Recorded By:</th>\n      <td>Egyptian astronomers</td>\n    </tr>\n  </tbody>\n<

In [126]:
## LETS GET THE MARS hemisphere images

browser = init_browser()

url = "https://astrogeology.usgs.gov/search/results?q=hemisphere+enhanced&k1=target&v1=Mars"
browser.visit(url)

time.sleep(1)

# Scrape page into Soup
html = browser.html
soup = bs(html, "html.parser")

In [131]:
## Find the image section of the page to iterate thru
results = browser.find_by_css(".collapsible results .item")
results
#items = results.find_by_css(".collapsible results .item")
#items

In [136]:
title = soup.find("div", class_="description")
title = title.find("h3").text
title

'Cerberus Hemisphere Enhanced'

In [ ]:
## Loop through img results and append results to a dict
img_dict = {}
counter = [-1]
for item in results:
    title = soup.find("div", class_="description")
    img_dict["title"] = title.find("h3").text
    
    browser.find_by_css(".full-content .thumb").click()
    time.sleep(1)
    browser.links.find_by_text("Sample").click()
    img_url = browser.url
    
    
    img_dict["img_url"] = img_url
    

    # Close the browser after scraping
    browser.quit()

In [ ]:
    
    
    
    
    
    
    # Store data in a dictionary
    image_dict = image_dict
    
    mars_data = {
        "news_title": news_title,
        "news_p": news_p,
        "featured_image_url": featured_image_url,
        "mars_facts": mars_facts,
        "img_list": img_list
    }


    return mars_data